# **`DSML_WS_11` - Decision Trees**

In this workshop we take a deep-dive on tree-based methods (and ensembles thereof) commonly used in a myriad of classification and regression problems.

We will cover the following: 
1. **Task**: Classifying breast cancer samples
1. **Task: Decision Trees for regression:** Predicting peak electricity demand
1. **Decision Trees for classification**: Classifying breast cancer cells
1. **Ensemble methods**: XGBoost, random forest

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
np.set_printoptions(suppress=True)

%matplotlib inline

---

## **1. Task: Classifying breast cancer samples**


In ``Workshop 9``, we looked at the workings of relevant classification algorithms. One issue with we did not consider was that we have trained our algorithms on the full set of available data. While this is fine for understanding how classification works in general, it is not suitable for developing predictive models (as you know by now).

As a result, the classification metrics from last week's workshop are relatively meaningless as we need to evaluate on previously unseen data. 

**Design a proper model development routine to train a high-performing classification algorithm for the breast cancer dataset. Proceed as follows:**

- Define your feature (let's continue to focus on `area_mean` and `concave points_mean`) and target sets.
- Partition the data into training, validation and test set, and scale the input features.
- Train a support vector machine on the training set.
- Tweak hyperparameters (e.g., kernel) by validating on the validation set.
- Report test metrics from the unseen test set (only look at the test set once you are finished validating your model. Do not go back and forth as this would create leakage!).

In [ ]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
# load data
cancer_df = pd.read_csv("breast_cancer.csv", index_col = "id")

# define feature and target
x = cancer_df[['area_mean','concave points_mean']].values
y = cancer_df['diagnosis'].values

# partition data: 50% Train, 20% Validation, 30% Test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)
X_train, X_hold, y_train, y_hold = train_test_split(X_train, y_train, test_size=(0.2/0.7),random_state=42)

# scale input features (based on X_train)
norm = StandardScaler()
X_train_norm = norm.fit_transform(X_train)      # 284 Observations
X_hold_norm = norm.transform(X_hold)            # 114 Observations
X_test_norm = norm.transform(X_test)            # 171 Observations

# For rounding digits in test metrics
Digits = 3

We evaluate three `support vector machine` - model specifications that minimize $\ell_{hinge}$-loss with different kernels:
- $SVM_{linear}$: Linear hypothesis function $h_{\theta}(x)$
- $SVM_{poly}$: Linear hypothesis function $h_{\theta}(x)$ with polynomial features
- $SVM_{RBF}$: Linear hypothesis function $h_{\theta}(x)$ with Radial Basis Functions

In [ ]:
# train model on training set
model_linear = SVC(kernel='linear', coef0=1.0)
model_linear.fit(X_train_norm, y_train)

# evaluate on holdout set
holdout_predictions = model_linear.predict(X_hold_norm)
print("Holdout Accuracy =", round(accuracy_score(y_hold, holdout_predictions),Digits))
print("Holdout Precision =", round(precision_score(y_hold, holdout_predictions, pos_label="M"),Digits))
print("Holdout Recall =", round(recall_score(y_hold, holdout_predictions, pos_label="M"),Digits))

In [ ]:
# train model on training set
model_poly = SVC(kernel='poly', C = 100, degree=2, coef0=1.0)   # Linear Hypothesis with Polynomial Features
model_poly.fit(X_train_norm, y_train)

# evaluate on holdout set
holdout_predictions = model_poly.predict(X_hold_norm)
print("Holdout Accuracy =", round(accuracy_score(y_hold, holdout_predictions),Digits))
print("Holdout Precision =", round(precision_score(y_hold, holdout_predictions, pos_label="M"),Digits))
print("Holdout Recall =", round(recall_score(y_hold, holdout_predictions, pos_label="M"),Digits))

In [ ]:
# train model on training set
model_RBF = SVC(kernel='rbf', C = 100, coef0=1.0)           # Linear Hypothesis with RBF Features
model_RBF.fit(X_train_norm, y_train)

# evaluate on holdout set
holdout_predictions = model_RBF.predict(X_hold_norm)
print("Holdout Accuracy =", round(accuracy_score(y_hold, holdout_predictions),Digits))
print("Holdout Precision =", round(precision_score(y_hold, holdout_predictions, pos_label="M"),Digits))
print("Holdout Recall =", round(recall_score(y_hold, holdout_predictions, pos_label="M"),Digits))

A comparison of the classification metrics shows that:
- Accuracy: $SVM_{RBF}$ < $SVM_{linear}$ < $SVM_{poly}$
- Precision: $SVM_{RBF}$ < $SVM_{linear}$ < $SVM_{poly}$
- Recall: $SVM_{RBF}$ < $SVM_{linear}$ $\leq$ $SVM_{poly}$

Hence, we choose $SVM_{poly}$ and report the final test metrics by evaluation of the test-set:

In [ ]:
# evaluate preferred model (SVM with polynomial kernel, d=2, C=100) on test set
test_predictions = model_poly.predict(X_test_norm)
print("Test Accuracy =", round(accuracy_score(y_test, test_predictions),Digits))
print("Test Precision =", round(precision_score(y_test, test_predictions, pos_label="M"),Digits))
print("Test Recall =", round(recall_score(y_test, test_predictions, pos_label="M"),Digits))

---

## **2. Task: Decision Trees for regression: Predicting peak electricity demand**

Revisit the tree-based regression introduced in ``Workshop 8``. In that session, we used a tree-based model to predict peak electricity demand based on temperature, splitting the data into training and testing sets. We also identified the optimal tree depth using a simple grid search over various tree depths. 

Instead of solely plotting the predicted regression line across all data in a scatterplot, we can also visually illustrate the model's desicion process using a tree diagram. This can be easily achieved with the ``plot_tree`` function from the ``sklearn.tree`` library.
Let's look at a very simple tree architecture for the prediciton of peak electricity demand - to do this, proceed as follows:

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, plot_tree
%matplotlib inline

**Step 1:** Load the ``Pittsburgh_load_data.csv`` dataset and specify the input vectors for your model:
- *Peak electricity demand* (``MAX``) as the prediction target
- *High temperature* (``High_temp``) as the predictor

In [ ]:
df = pd.read_csv("Pittsburgh_load_data.csv")

xp = df["High_temp"]
yp = df["MAX"]

**Step 2:** Use the ``DesicisonTreeRegressor`` to fit a decision tree of max_depth = 2 to your data and plot the results in a scatterplot. Name the model *tree_model*.

***Note:** For demonstration purposes, skip the training, validation and test splits, that would normally be required for prediction (as done in the first task). Our goal in this introductory setting is simply to get started with the visualization of tree diagrams and their interpretation.*

In [ ]:
# initialize
Tree_reg = DecisionTreeRegressor(max_depth=2,
                                 criterion="squared_error")

# train
tree_model = Tree_reg.fit(xp.values.reshape((-1,1)), yp)

# Further information on the fitted tree
attributes = tree_model.tree_

# plot
plt.figure(figsize = (8,6))
plt.scatter(xp, yp, marker="x")
plt.plot(np.arange(-18,40,1), Tree_reg.predict(np.arange(-18,40,1).reshape((-1,1))), marker="x", color='C1')
plt.xlabel("High Temperature (°C)")
plt.ylabel("Peak Demand (GW)")
plt.show()

print("Number of nodes: ", attributes.node_count)
print("Number of leafs: ", attributes.n_leaves)

**Step 3:** Visualize the tree diagram, by utilizing the ``plot_tree`` function.

Start by initializing a figure with ``plt.figsize = (12,6)``, then use ``plot_tree(tree_model, feature_names=['High Temperature'])``.
- Can you interpret the values that are depicted in the graphical representation of the tree?
- How would you traverse the tree to predict peak electricity demand of a day with a high temperature of 30 C°?

In [ ]:
plt.figure(figsize = (12,6))
plot_tree(tree_model, feature_names=['High Temperature'])
plt.show()

print("A temperature of 30 C° would lead to peak electricity demand of approx.", round(tree_model.predict([[30]])[0],2), "GW.")

---

## **3. Decision Trees for classification: Classifying breast cancer cells**

In [ ]:
# load dataset
cancer_df = pd.read_csv("breast_cancer.csv", index_col="id")
cancer_df.head()

To abstract from the relatively high-dimensionality of the breast cancer dataset let us confine our analysis to a two-dimensional feature vector consisting of `area_mean` and `concave points_mean` for now.

In [ ]:
# Define a function for easier plotting
def plot_cells():
    plt.figure(figsize=(8,6))
    plt.scatter(cancer_df[cancer_df["diagnosis"]=='M']['area_mean'], cancer_df[cancer_df["diagnosis"]=='M']['concave points_mean'], marker='x', color='C3')
    plt.scatter(cancer_df[cancer_df["diagnosis"]=='B']['area_mean'], cancer_df[cancer_df["diagnosis"]=='B']['concave points_mean'], marker='+', color='C0')
    plt.xlim([0,2600])
    plt.ylim([0,0.21])
    plt.xlabel("Mean Area",fontsize=16)
    plt.ylabel("Mean Concave Points",fontsize=16)
    plt.legend(['Malignant','Benign'],fontsize=12)
    
plot_cells()
plt.show()

# Confine the dataset to the features 'area_mean' and 'concave points_mean'
X = np.array(cancer_df[['area_mean','concave points_mean']])
Y = cancer_df['diagnosis'].values

# recode Y to 0 and 1
Y  = np.where(Y=="M", int(1), Y) # Malign = 1
Y  = np.where(Y=="B", int(0), Y) # Benign = 0
Y = Y.astype('int')

### **Applying the DecisionTreeClassifier and accessing its components**

Let's specify and fit a simple `DecisionTreeClassifier`, which is available via `sklearn`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

tree_classifier = DecisionTreeClassifier(max_depth=2,criterion='gini') # we set gini as our impurity measure
tree_classifier.fit(X, Y)

The decision estimator has an attribute called tree_  which stores the entire tree structure and allows access to low-level attributes. The binary tree_ attribute is represented as a number of parallel arrays. The i-th element of each array holds information about the node `i`. Node 0 is the tree's root.

In [ ]:
structure = tree_classifier.tree_         # All information to draw the tree is available through the tree_ function

<div class="alert alert-block alert-info">
<b>Question:</b> Recall the basic terminology for decision trees, including nodes, leaves, parent and children nodes, threshold and impurity: What do they represent?
</div>

Among those arrays, we have:

   - ``left_child``: ID of the left child of the node
   - ``right_child``: ID of the right child of the node
   - ``feature``: Feature used for splitting the node
   - ``threshold``: Threshold value used for splitting the node
   - ``impurity``: The impurity at the node
   - etc.

In [ ]:
# assign various tree attributes
n_nodes = structure.node_count
n_leaves = structure.n_leaves
children_left = structure.children_left
children_right = structure.children_right
feature = structure.feature
threshold = structure.threshold
impurity = structure.impurity

In [ ]:
print("Num nodes: \t",n_nodes)        # How many nodes does the tree have?
print("Num leaves: \t",n_leaves)      # How many leaves does the tree have?

<div class="alert alert-block alert-info">
<b>Question:</b> Based on this information, can you draw the basic structure of the tree?
</div>

In [ ]:
print("left children per node: ", children_left)      # The seven values here correspond to the seven nodes above.
print("right children per node: ", children_right)

<div class="alert alert-block alert-info">
<b>Question:</b> Based on this information, can you add the node IDs to your basic tree sketch?
</div>

In [ ]:
print("Decision feature at node: ", feature)          # Features area mean (0) and concave points (1) have been encoded to 0 and 1. -2 descibes a leaf
print("Threshold of feature at node", threshold)
print("Impurity at node: ", impurity)                 # Impurity decreases as we run down the tree: Leaf nodes are more homogeneous

<div class="alert alert-block alert-info">
<b>Question:</b> Based on this information, can you derive which feature and threshold was used to split each node?
</div>

### **Retrieval of decision paths: An Example**

The `decision_path` method allows us to retrieve the node indicator functions. A non-zero element of an indicator matrix at the position (i, j) indicates that the sample i goes through the node j.

- ``.indices`` stores all nodes that are visited by the sample [sample_id]
- ``.indptr`` tells us, where the start [sample_id]- and end-nodes [sample_id+1] are in the .indices array

In [ ]:
# Decision_path Results in a 569x7 sparse matrix: n_samples x n_nodes
node_indicators = tree_classifier.decision_path(X)     # Each row shows, which nodes were visited by the sample

# let's generate a random sample ID                    Note: sample ID is a specific combination of features in our feature matrix
sample_id = 10

# retrieve decision_path for that sample:
node_index = node_indicators.indices[node_indicators.indptr[sample_id] : node_indicators.indptr[sample_id + 1]]  #indptr maps the elements of data and indices to the rows of the sparse matrix

print("Value of Feature ``Area Mean`` for Sample ID", sample_id ,"in Array:", X[sample_id,0])
print("Value of Feature ``Concave Points`` for Sample ID", sample_id ,"in Array:", X[sample_id,1])
print("Decision path for sample " + str(sample_id), ": ", str(node_index))

<div class="alert alert-block alert-info">
<b>Question:</b> For our basic decision tree, what combinations of IDs could appear as decision paths?
</div>

The `apply` method can be used to get the index of the leaf (Terminal Node) that each sample is predicted as from the incorporated features.

In [ ]:
# we can also retrieve the leaf ids reached by each sample using .apply
leaf_ids = tree_classifier.apply(X)

leaf_ids[:10]           # i.e. the terminal (leaf) node: Predicition, which our Sample IDs "end up in"

<div class="alert alert-block alert-info">
<b>Question:</b> What IDs could be returned by apply()?
</div>

Finally, let us also consider the features and thresholds that were used to predict a certain sample.

In [ ]:
print('Decision path for sample %s: %s' % (str(sample_id), str(node_index)))
print('Feature values of sample %s: %s \n' % (sample_id, X[sample_id]))
print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:                              # Recall: node_index stores nodes that are traversed by sample_id
    # skip leaf node
    if leaf_ids[sample_id] == node_id:
        continue
    
    # for all other nodes, retrieve the feature values
    if (X[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("Decision at node %s: value for feature %s (%s) is %s the threshold of %s"
          % (node_id,
             feature[node_id],
             X[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

### **Plotting the full decision tree**

In [ ]:
from sklearn import tree

plt.figure(figsize=(10,6))
tree.plot_tree(tree_classifier,
               class_names=['Malignant','Benign'],
               feature_names=['area_mean', 'concave points_mean'])   # Malignant = 1, Benign = 0
plt.show()   # Classification model: We use the majority vote to determine a nodes class. Default cutoff = 0.5

In the decision tree each node is represented by a box. For each node the following information is provided:
- decision feature and threshold
- impurity
- number of samples
- number of samples per class
- class (i.e., majority vote)

We can, thus, easily relate this back to the tree attributes we computed above. A selection is below:

In [ ]:
print("Num nodes: \t",n_nodes)
print("Num leaves: \t",n_leaves)
print("Feature at node", feature) # -2 indicates no feature/threshold, i.e. a leaf
print("Threshold of feature at node", threshold)
print("Impurity at node: ", impurity)

### **Plotting decision surfaces**

As we have seen in the lecture, another intuitive representation of decision trees is the use of decision surfaces. These can be related back directly to the decision tree. For ease of use, a plotting routine has been prepared that combines fitting and plotting into a single routine and allows for easy adjustment of tree depth and the minimum samples per leaf (discussed below).

In [ ]:
def plot_class_surface(max_depth,min_samples_leaf=1):    # The number inserted here determines the amount of leaves
    
    # specify and fit decision tree classifier
    from sklearn.tree import DecisionTreeClassifier, export_graphviz # we also call the garphviz module for later visualization
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_leaf=min_samples_leaf,
                                  criterion='gini') # we set entropy as our impurity measure
    model.fit(X, Y)
    
    # get tree attributes
    attributes = model.tree_
    
    # define range per feature
    x_range = [0,2600] # i.e. mean area
    y_range = [0, 0.21] # i.e mean conc. points
    plt.figure(figsize=(8,6))
    
    # plot classification regions
    grid=1000
    xx,yy = np.meshgrid(np.linspace(x_range[0], x_range[1], grid),
                        np.linspace(y_range[0], y_range[1], grid))

    zz = model.predict(np.c_[xx.ravel(), yy.ravel()])
    zz = zz.reshape(xx.shape)
    cs = plt.contourf(xx, yy,zz,levels=[-float("inf"),0,float("inf")],alpha=0.2,colors=["b","r"])
    plt.contour(cs, colors='k')
    
    # plot data points
    s1 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='M']['area_mean'], cancer_df[cancer_df["diagnosis"]=='M']['concave points_mean'], marker='x', color='C3')
    s2 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='B']['area_mean'], cancer_df[cancer_df["diagnosis"]=='B']['concave points_mean'], marker='+', color='C0')    
    plt.xlim([0,2600])
    plt.ylim([0,0.21])
    plt.xlabel("Mean Area",fontsize=16)
    plt.ylabel("Mean Concave Points",fontsize=16)
    plt.legend([s1,s2],['Malignant','Benign'],fontsize=12)
    
    print("number of nodes: ", attributes.node_count)
    print("number of leafs: ", attributes.n_leaves)
    
    plt.show()
    
    #plt.savefig("Breast_Cancer_Decision_Surface_{}depth.pdf".format(tree_depth))

In [ ]:
plot_class_surface(max_depth=2, min_samples_leaf=1)

### **Controlling overfitting in Decision Trees**

**Decision-tree learners can create overly complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.**

This can easily be seen by increasing tree depth to unreasonable values:

In [ ]:
plot_class_surface(max_depth=15, min_samples_leaf=1)     # Tree reaches terminal state eventually "is fully grown"

What can we do about overfitting in sklearn? As mentioned, we have several tools at our disposal:
- **max_depth**: The maximum depth of the tree. If None, then nodes are expanded until all the leaves contain less than min_samples_split samples. A too high value of maximum depth causes overfitting, whereas a too low value causes underfitting.
- **min_samples_leaf**: By specifying a minimum number of samples per leaf, overfitting can be controlled for.
- **ccp_alpha**: Cost Complexity (CCP) alpha paramter determining the size of the penalty.

<div class="alert alert-block alert-info">
<b>Question:</b> Check the effect of the max_depth and the min_samples_leaf parameters. How would you use them to prevent overfitting?
</div>

In [ ]:
plot_class_surface(max_depth = 15, min_samples_leaf = 1)
# increasing min_samples_leaf is decreasing tree size: Each Node needs to have a specific number of observations remaining
# decreasing max_depth directly prunes the tree, counteracting overfitting

### **Cost Complexity of a Decision Tree: Effective Alphas**

Let us look at the **cost complexity** as an effective measure in avoiding overfitting. The cost complexity *CCP(T)* of a tree  is defined as 

\begin{equation}
CCP(T) = ERR(T) + \alpha \cdot L(T)
\end{equation}

where
- **ERR(T)** is the total misclassification rate of the terminal nodes
- **L(T)** is the number of terminal nodes of tree T, also called *"tree complexity"*
- **$\alpha$** is a penalty parameter  related  to tree complexity L(T).

This type of formula should look familiar, as it closely resembles the regularized regression loss functions we know.

To get an idea of what values of $\alpha$ could be appropriate, `scikit-learn` provides `DecisionTreeClassifier.cost_complexity_pruning_path`, that returns the **effective $\alpha$ values** (i.e. the values of $\alpha$, that will achieve the next step in complexity reduction) and the corresponding total leaf impurities at each step of the pruning process. As $\alpha$ increases, more of the tree is pruned, which increases the total impurity of its leaves.

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

# fit decision tree (without limit on max_depth, i.e. tree will grow fully if alpha is set to 0)
tree_classifier = DecisionTreeClassifier(random_state=0, 
                                         criterion="gini")

# compute cost_complexity_pruning_path 
path = tree_classifier.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

pd.DataFrame({"Effective Alpha": ccp_alphas,
             "Gini-Index": impurities})

``ccp_alphas`` stores a list of alphas, which lead to the next pruning step. ``impurities`` gives out the values of the chosen homogeneity-measure.

We can visualize this in a graph:

In [ ]:
#plot cost_complexity_pruning_path
fig, ax = plt.subplots(figsize=(8,6)) # plot cccp alphas against impurities
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")  # we remove the last alpha as this corresponds to a tree with only the root node
ax.set_xlabel("effective alpha",fontsize=16)
ax.set_ylabel("total impurity of leaves",fontsize=16)
ax.set_title("Total Impurity vs effective alpha for training set",fontsize=16)
plt.show()
#plt.savefig("Determining_Alpha.pdf")

Next, we train a decision tree using the effective alphas. The last value in ccp_alphas is the alpha value that prunes the whole tree, leaving the tree with one node.

In [ ]:
trees = []
for ccp_alpha in ccp_alphas:
    tree = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    tree.fit(X_train, y_train)
    trees.append(tree)
print("Number of nodes in the last tree is {} with ccp_alpha: {}".format(
      trees[-4].tree_.node_count, ccp_alphas[-4]))     # If changed to -2, this is a two layer tree

For the remainder of this example, we remove the last element in clfs and ccp_alphas, because it is the trivial tree with only one node.\
Here we show that the number of nodes and tree depth decreases as alpha increases.

In [ ]:
# Here, we are showing the pruning effect on the tree
trees = trees[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [tree.tree_.node_count for tree in trees]
depth = [tree.tree_.max_depth for tree in trees]
fig, ax = plt.subplots(1,2,figsize=(14,6))
ax[0].plot(ccp_alphas, node_counts, marker='o', drawstyle="steps-post")
ax[0].set_xlabel("alpha",fontsize=16)
ax[0].set_ylabel("number of nodes",fontsize=16)
ax[0].set_title("Number of nodes vs alpha",fontsize=16)
ax[1].plot(ccp_alphas, depth, marker='o', drawstyle="steps-post")
ax[1].set_xlabel("alpha",fontsize=16)
ax[1].set_ylabel("depth of tree",fontsize=16)
ax[1].set_title("Depth vs alpha",fontsize=16)
fig.tight_layout()
plt.show()
#plt.savefig("Pruning_effect.pdf")

As we increase alpha, we penalize more complex tree structures. Hence, the number of nodes and the depth of the tree decreases.

### **Effective Alphas and Predictive Performance**

Now, we can implement a grid search over the identified effective alphas to determine where predictive performance is maximized.

In [ ]:
scores_train = []
scores_val = []

# run loop
for ccp_alpha in ccp_alphas:

    model = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    model.fit(X_train, y_train)

    from sklearn.metrics import accuracy_score
    accuracy_train = accuracy_score(y_train, model.predict(X_train))
    accuracy_val = accuracy_score(y_test, model.predict(X_test))
    scores_val.append(accuracy_val)
    scores_train.append(accuracy_train)

# collect results in DF
df = pd.DataFrame(columns=["alpha", "train_score", "test_score"])
df["alpha"] = ccp_alphas
df["train_score"] = scores_train
df["test_score"] = scores_val

# show top five alphas
df.sort_values("test_score", ascending=False).head(5)

In [ ]:
plt.subplots(figsize = (8,6))
plt.plot(df["alpha"], df["train_score"], marker = 'o')
plt.plot(df["alpha"], df["test_score"], marker = 'o')
plt.legend(["train", "test"])
plt.xlabel("effective alpha", fontsize =  16)
plt.ylabel("accuracy", fontsize =  16)
plt.title("Accuracy for different alphas", fontsize = 16)
plt.show()

#plt.savefig("alpha_tuning.pdf")

Note, that the y-axis reflects accuracy, not error! The *best* model is hence achieved with $\alpha \approx 0.004$.

## **-- Small excursion: Naive Bayes --**

In last week's lecture, you also learned about the Naive Bayes algorithm. It is based on conditional probabilities that you use to calculate the change in probability of class membership. Say, for example, you have an unknown cell structure and want to calculate how likely it is that this cell belongs to the Malignant cells. You can, then, look at the conditional probability of a cell having that specific mean area to predict whether the unknown cell is malignant.

In [ ]:
# use Naive Bayes from sklearn
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()                     # Gaussian Naive Bayes assumes that each feature follows a Gaussian normal distribution indepently (!) for each class
gnb.fit(X_train, y_train)

point_benign, point_malignant = gnb.theta_              # gives the mean of each feature per class: The center of the gaussian
radius_benign, radius_malignant = np.sqrt(gnb.var_)     # gives the SD of each feature per class: The spread around the mean

In [ ]:
import matplotlib.patches as mpl_patches
benign = mpl_patches.Ellipse(xy=point_benign, width=3 * radius_benign[0], height=3 * radius_benign[1], facecolor='none', edgecolor='b', linewidth=0.5)       # Multiplication with 3 Standard Deviation includes 99.7% of the data, assuming gaussian distribution
malignant = mpl_patches.Ellipse(xy=point_malignant, width=3 * radius_malignant[0], height=3 *radius_malignant[1], facecolor='none', edgecolor='r', linewidth=0.5)
plt.scatter(point_benign[0], point_benign[1], color='b')
plt.scatter(point_malignant[0], point_malignant[1], color='r')

ax = plt.gca()
ax.add_artist(benign)
ax.add_artist(malignant)

s1 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='M']['area_mean'], cancer_df[cancer_df["diagnosis"]=='M']['concave points_mean'], marker='x', color='C3', linewidths=0.2)
s2 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='B']['area_mean'], cancer_df[cancer_df["diagnosis"]=='B']['concave points_mean'], marker='+', color='C0', linewidths=0.2)    
plt.xlim([0,2600])
plt.ylim([0,0.21])
plt.xlabel("Mean Area",fontsize=16)
plt.ylabel("Mean Concave Points",fontsize=16)
plt.legend([s1,s2],['Malignant','Benign'],fontsize=12)
plt.show()

You could now evaluate and compare both models using the known cross-validation procedure and classification evaluation metrics.

---

## **4. Ensemble Methods**

In predictive modeling, “risk” is equivalent to variation (i.e. variance) in prediction error. Ensemble methods are targeted at reducing variance, thus increasing predictive power.
The core idea is that by combining the outcomes of individual models, e.g., by taking an average, variance may be reduced. Thus, using an average of two or more predictions can potentially lead to smaller error variance, and therefore better predictive power.

We will not discuss ensemble methods in detail here, but will limit our discussion to a brief introduction of two very popular tree-based ensemble methods. These are:

- RandomForest: see [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
- XGBoost (a type a boosting method): see [here](https://xgboost.readthedocs.io/en/latest/)

### **Bagging:** Random Forest

Random Forests is a selection of n trees which are trained in parallel. Predictions are made by averaging the outputs across these n trees. Random Forest are most often combined with **bagging** (**B**ootstrap **agg**regat**ing**), i.e. different boostrap samples of the training data are used to train the individual trees.
- Each tree is trained on a different bootstrap sample of training data
- At each node in a tree, it only looks at random subsets of features when choosing the best fit

In [ ]:
# train test split on breast cancer dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# sepcify and fit model
rf_classifier = RandomForestClassifier(n_estimators=100, 
                                       bootstrap=True, random_state=42) # we select boostrapp, i.e. we use bagging
R_FOREST = rf_classifier.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test,R_FOREST.predict(X_test))

In [ ]:
accuracy_score(y_test,R_FOREST.predict(X_test))

Just by taking the default setting, we obtain very good results that are comparable to those of the fully grid-searched decision tree.

### **Boosting:** XGBoost

XGBoost is an ensemble method that uses **boosting**. While XGBoost is not included in sklearn, there is a very well developed API that can be installed by executing the following command:
- `conda install -c conda-forge xgboost`

Once you have completed the installation you are good to go. Let us fit a very simple classifier to the breast cancer dataset.

XGBoost is a specific implementation of boosting that is known for being very fast and efficient. The trees are trained independently, where each tree is trained to fix the errors made by the previous trees.

In [ ]:
# specify and fit model
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(booster="gbtree")
XGBOOST = xgb_classifier.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test,XGBOOST.predict(X_test))

In [ ]:
accuracy_score(y_test,XGBOOST.predict(X_test))

Obviously, there is likely room for improvement as you grid search some of the hyperparameters. However, by just taking the default setting, we already achieve an accuracy score that is comparable to that of the grid-searched decision tree above.

---